In [1]:
import gymnasium as gym
import pygame
import numpy as np
import time
import tensorflow as tf
from tensorflow import keras

In [2]:
env = gym.make("CartPole-v1", render_mode='human')
#observatoin:
# [0]: horizontal position (0.0 is center)
# [1]: velocity (positive means going right)
# [2] angle of the pole ( 0.0 is verical)
# [3] angular velocity ( positive means clockwise)
print('observation space = ', env.observation_space)
# 2 actions: 0 left, 1 right
print('action space = ', env.action_space)
env.reset()

observation space =  Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
action space =  Discrete(2)


(array([ 0.01952637, -0.03450128, -0.04079926,  0.02818657], dtype=float32),
 {})

In [ ]:
#Sample render
done = False
env.reset()
while not done:
    obs, reward, done, terminated, info = env.step(0)
    print(obs.shape)
    print(obs[np.newaxis].shape)
    print()
    time.sleep(0.5)

In [3]:
def simple_policy(obs):
    angle = obs[2]
    return 0 if angle < 0 else 1

totals = []

print('simple policy')
for ep in range(500):
    ep_reward = 0.0
    obs = env.reset()
    obs = obs[0]

    for step in range(200):
        action = simple_policy(obs) 
        obs, reward, done, terminated, info = env.step(action)
        ep_reward += reward
        if done:
            break
        
    totals.append(ep_reward)

print(totals)
    

simple policy


KeyboardInterrupt: 

In [5]:
def make_model():
    #there are 4 dimension in env state
    model = keras.models.Sequential([
        keras.layers.Dense(5, activation='elu', input_shape = 4),
        keras.layers.Dense(1, activation='sigmoid'),
    ])
    return model

In [10]:
def play_one_step(env, obs, model: keras.models.Sequential, loss_fn):

    with tf.GradientTape() as tape:
        left_prob = model(obs[np.newaxis])
        action = (tf.random.uniform([1,1]) > left_prob)
        y_target = tf.constant([1.]) - tf.cast(action, tf.float32)
        loss = tf.reduce_mean(loss_fn(y_target, left_prob))
    
    grads = tape.gradient(loss, model.trainable_variables)
    obs, reward, done, info = env.step(int(action[0,0].numpy()))
    return obs, reward, done, info


In [17]:
def play_muliple_eps(env, n_episodes, n_max_steps, model, loss_fn):
    all_rewards = []
    all_grads = []

    for ep in range(n_episodes):
        curr_rewards = []
        curr_grads = []

        obs = env.reset()

        for step in range(n_max_steps):
            obs, reward, done, grads = play_one_step(env, obs, model, loss_fn)
            curr_rewards.append(reward)
            curr_grads.append(grads)

            if done: break
            
        all_rewards.append(curr_rewards)
        all_grads.append(curr_grads)
    
    return all_rewards, all_grads

In [ ]:
def discount_rewards(rewards, discount_factor):
    discounted = np.array(rewards)
    for step in range(len(discounted) - 2, -1, -1):
        discounted[step] += discounted[step + 1] * discount_factor
    return discounted
